In [1]:
from bokeh.plotting import figure, show, output_notebook
from bokeh.layouts import row
from bokeh.io import push_notebook
from ipywidgets import interact

import numpy  as np
import math

output_notebook()

Loading BokehJS ...

# Functions-utils

In [2]:
def count_f(x, y):
    return x * y * y + 3 * x * y

In [3]:
def count_exact(x):
    return -(3*np.e**(3/2*x**2))/(np.e**(3/2*x**2)-2)

In [4]:
def count_k(x, y, h):
    k1 = h*count_f(x, y)
    k2 = h*count_f(x + h / 2, y + k1 / 2)
    k3 = h*count_f(x + h / 2, y + k2 / 2)
    k4 = h*count_f(x + h, y + k3)
    return 1 / 6 * (k1 + 2 * k2 + 2 * k3 + k4)

# Exact solution

In [5]:
def exact(x0, X):
    # step for drawing
    h = 0.0001
    # find x position of asymptote
    xap =  math.sqrt(2/3*np.log(2))
    xan = -xap
    # Every x and y lists are for each interval of domain
    y1 = []
    y2 = []
    x1 = []
    x2 = []
    x3 = []
    y3 = []
    counter = int((X-x0)/h)
    # in this loop arrays are filled with needed values
    for i in range(counter):
        if x0 < xan:
            x1.append(x0)
            y1.append(count_exact(x0))
        elif x0 > xan and x0 < xap:
            x2.append(x0)
            y2.append(count_exact(x0))
        elif x0 > xap:
            x3.append(x0)
            y3.append(count_exact(x0))
        x0 =x0+h 
    return [x1, y1, x2, y2, x3, y3]
    

# Euler method

In [6]:
def euler_method(x0, X, y0, h):
    # find x position of asymptote
    xap = math.sqrt(2/3*np.log(2))
    xan = -xap
    # Every x and y lists are for each interval of domain
    y1 = []
    y2 = []
    x1 = []
    x2 = []
    x3 = []
    y3 = []
    f1 = []
    f2 = []
    f3 = []
    counter = int((X-x0)/h)
    # fill the array with initial values 
    if x0 < xan:
            x1.append(x0)
            y1.append(y0)
            f1.append(count_f(x0, y0))
    elif x0 > xan and x0 < xap:
            x2.append(x0)
            y2.append(y0)
            f2.append(count_f(x0, y0))
    elif x0 > xap:
            x3.append(x0)
            y3.append(y0)
            f3.append(count_f(x0, y0))
    # fill the array with values within required interval
    for i in range(counter):
        if x0 < xan:
            x1.append(x1[len(x1) - 1] + h)
            y1.append(y1[len(y1) - 1] + h * f1[len(f1) - 1])
            f1.append(count_f(x1[len(x1) - 1], y1[len(y1) - 1]))
        elif x0 > xan and x0 < xap:
            # handling asymptots
            if len(y2) == 0:
                x0 = -0.6
                x2.append(x0)
                y2.append(count_exact(x0))
            else:
                x2.append(x0)
                y2.append(y2[len(y2) - 1] + h * f2[len(f2) - 1])
            f2.append(count_f(x2[len(x2) - 1], y2[len(y2) - 1]))
        elif x0 > xap:
            # handling asymptots
            if len(y3) == 0:
                x0 = 0.78
                x3.append(x0)
                y3.append(count_exact(x0))
            else:
                x3.append(x0)
                y3.append(y3[len(y3) - 1] + h * f3[len(f3) - 1])
            f3.append(count_f(x3[len(x3) - 1], y3[len(y3) - 1]))
        x0 =x0+h 
    return [x1, y1, x2, y2, x3, y3]


# Improved euler method

In [7]:
def improved_euler_method(x0, X, y0, h):
    # find x position of asymptote
    xap = math.sqrt(2/3*np.log(2))
    xan = -xap
    # Every x, y, and f lists are for each interval of domain
    y1 = []
    y2 = []
    x1 = []
    x2 = []
    x3 = []
    y3 = []
    f1 = []
    f2 = []
    f3 = []
    yp1 = []
    yp2 = []
    yp3 = []
    counter = int((X-x0)/h)
    # fill the array with initial values 
    if x0 < xan:
            x1.append(x0)
            y1.append(y0)
            f1.append(count_f(x0, y0))
    elif x0 > xan and x0 < xap:
            x2.append(x0)
            y2.append(y0)
            f2.append(count_f(x0, y0))
    elif x0 > xap:
            x3.append(x0)
            y3.append(y0)
            f3.append(count_f(x0, y0))
    # fill the array with values within required interval
    for i in range(counter):
        if x0 < xan:
            x1.append(x1[len(x1) - 1] + h)
            yp1.append(y1[len(y1) - 1] + h*f1[len(f1) - 1])
            y1.append(y1[len(y1) - 1] + h * (
                        f1[len(f1) - 1] + count_f(x1[len(x1) - 1],
                                                            yp1[len(yp1) - 1])) / 2)
            f1.append(count_f(x1[len(x1) - 1], y1[len(y1) - 1]))
        elif x0 > xan and x0 < xap:
            # handling asymptots
            if len(y2) == 0:
                x0 = - 0.6
                x2.append(x0)
                y2.append(count_exact(x0))
            else:
                x2.append(x0)
                yp2.append(y2[len(y2) - 1] + h*f2[len(f2) - 1])
                y2.append(y2[len(y2) - 1] + h * (
                        f2[len(f2) - 1] + count_f(x2[len(x2) - 1],
                                                            yp2[len(yp2) - 1])) / 2)
            f2.append(count_f(x2[len(x2) - 1], y2[len(y2) - 1]))
        elif x0 > xap:
            # handling asymptots
            if len(y3) == 0:
                x0 = 0.75
                x3.append(x0)
                y3.append(count_exact(x0))
            else:
                x3.append(x0)
                yp3.append(y3[len(y3) - 1] + h*f3[len(f3) - 1])
                y3.append(y3[len(y3)-1]+h*(
                        f3[len(f3) - 1] + count_f(x3[len(x3) - 1],
                                                            yp3[len(yp3) - 1])) / 2)
            f3.append(count_f(x3[len(x3) - 1], y3[len(y3) - 1]))
        x0 =x0+h 
    return [x1, y1, x2, y2, x3, y3]

# Runge-Kutta method

In [8]:
def runge_kutta(x0, X, y0, h):
    #finding x of asymptots
    xap = math.sqrt(2/3*np.log(2))
    xan = -xap
    # Initialise lists for all intervals
    y1 = []
    y2 = []
    x1 = []
    x2 = []
    x3 = []
    y3 = []
    counter = int((X-x0)/h)
    # Setting initial values
    if x0 < xan:
            x1.append(x0)
            y1.append(y0)
    elif x0 > xan and x0 < xap:
            x2.append(x0)
            y2.append(y0)
    elif x0 > xap:
            x3.append(x0)
            y3.append(y0)
    
    # fill in the results in lists within each interval
    for i in range(counter):
        if x0 < xan:
            to_add = count_k(x1[len(x1) - 1], y1[len(y1) - 1], h)
            x1.append(x1[len(x1) - 1] + h)
            y1.append(y1[len(y1) - 1] + to_add)
           
        elif x0 > xan and x0 < xap:
            if len(y2) == 0:
                x0 = -0.6
                x2.append(x0)
                y2.append(count_exact(x0))
            else:
                x2.append(x0)
                to_add = count_k(x2[len(x2) - 1], y2[len(y2) - 1], h)
                y2.append(y2[len(y2) - 1] + to_add)
        elif x0 > xap:
            
            if len(y3) == 0:
                x0 = 0.75
                x3.append(x0)
                y3.append(count_exact(x0))
            else:
                x3.append(x0)
                to_add = count_k(x3[len(x3) - 1], y3[len(y3) - 1], h)
                y3.append(y3[len(y3) - 1] + to_add)
        x0 =x0+h 
    return [x1, y1, x2, y2, x3, y3]


# Plot graph

In [9]:
fig = figure(title = 'Numerical methods', x_axis_label ='x', y_axis_label='y', y_range =(-100, 100))
exact_results = exact(0, 5.5)
euler_results = euler_method(0,5.5, 3, 0.1)
improved_euler_results = improved_euler_method(0,5.5, 3, 0.1)
runge_kutta_results = runge_kutta(0, 5.5, 3, 0.1)

exact1 = fig.line(exact_results[0], exact_results[1], line_width =2, legend = "Exact solution")
exact2 = fig.line(exact_results[2], exact_results[3], line_width =2, legend = "Exact solution")
exact3 = fig.line(exact_results[4], exact_results[5], line_width =2, legend = "Exact solution")

euler1 = fig.line(euler_results[0], euler_results[1], line_width =2, color="black", legend = "Euler method")
euler2 = fig.line(euler_results[2], euler_results[3], line_width =2, color="black", legend = "Euler method")
euler3 = fig.line(euler_results[4], euler_results[5], line_width =2, color="black", legend = "Euler method")

improved_euler1 = fig.line(improved_euler_results[0], improved_euler_results[1], line_width =2, color="red", legend = "Improved Euler method")
improved_euler2 = fig.line(improved_euler_results[2], improved_euler_results[3], line_width =2, color="red", legend = "Improved Euler method")
improved_euler3 = fig.line(improved_euler_results[4], improved_euler_results[5], line_width =2, color="red", legend = "Improved Euler method")

runge_kutta1 = fig.line(runge_kutta_results[0], runge_kutta_results[1], line_width =2, color="green", legend = "Runge-Kutta method")
runge_kutta2 = fig.line(runge_kutta_results[2], runge_kutta_results[3], line_width =2, color="green", legend = "Runge-Kutta method")
runge_kutta3 = fig.line(runge_kutta_results[4], runge_kutta_results[5], line_width =2, color="green", legend = "Runge-Kutta method")

fig.legend.click_policy = "hide"



# Plot error

In [10]:
euler_error1 = []
euler_error2 = []
euler_error3 = []
improved_euler_error1 = []
improved_euler_error2 = []
improved_euler_error3 = []
runge_kutta_error1 = []
runge_kutta_error2 = []
runge_kutta_error3 = []
for i in range(len(euler_results[0])):
    euler_error1.append(abs(count_exact(euler_results[0][i])-euler_results[1][i]))
    improved_euler_error1.append(abs(count_exact(euler_results[0][i])-improved_euler_results[1][i]))
    runge_kutta_error1.append(abs(count_exact(euler_results[0][i])-runge_kutta_results[1][i]))
for i in range(len(euler_results[2])):
    euler_error2.append(abs(count_exact(euler_results[2][i])-euler_results[3][i]))
    improved_euler_error2.append(abs(count_exact(euler_results[2][i])-improved_euler_results[3][i]))
    runge_kutta_error2.append(abs(count_exact(euler_results[2][i])-runge_kutta_results[3][i]))
for i in range(len(euler_results[4])):
    euler_error3.append(abs(count_exact(euler_results[4][i])-euler_results[5][i]))
    improved_euler_error3.append(abs(count_exact(euler_results[4][i])-improved_euler_results[5][i]))
    runge_kutta_error3.append(abs(count_exact(euler_results[4][i])-runge_kutta_results[5][i]))

In [11]:
err = figure(title = 'ERROR', x_axis_label = 'x', y_axis_label = 'y')
euler_err1 = err.line(euler_results[0], euler_error1, line_width =2, color="black", legend = "Euler error")
euler_err2 = err.line(euler_results[2], euler_error2, line_width =2, color="black", legend = "Euler error")
euler_err3 = err.line(euler_results[4], euler_error3, line_width =2, color="black", legend = "Euler error")

improved_euler_err1 = err.line(improved_euler_results[0], improved_euler_error1, line_width =2, color="red", legend = "Improved Euler error")
improved_euler_err2 = err.line(improved_euler_results[2], improved_euler_error2, line_width =2, color="red", legend = "Improved Euler error")
improved_euler_err3 = err.line(improved_euler_results[4], improved_euler_error3, line_width =2, color="red", legend = "Improved Euler error")

runge_kutta_err1 = err.line(runge_kutta_results[0], runge_kutta_error1, line_width =2, color="green", legend = "Runge-Kutta error")
runge_kutta_err2 = err.line(runge_kutta_results[2], runge_kutta_error2, line_width =2, color="green", legend = "Runge-Kutta error")
runge_kutta_err3 = err.line(runge_kutta_results[4], runge_kutta_error3, line_width =2, color="green", legend = "Runge-Kutta error")

err.legend.click_policy = "hide"

# Interactive part

In [12]:
def update(new_x0 = '0', new_y0 = '3', new_X = '5.5', new_h = '0.1'):
    if new_x0 == '' or new_y0 == '' or new_X == '' or new_h == '':
        return
    if  new_x0 == '-' or new_y0 == '-' or new_X == '-' or new_h == '-':
        return
    if float(new_x0) > float(new_X) or float(new_h) <= 0:
        return
    x0 = float(new_x0)
    y0 = float(new_y0)
    X = float(new_X)
    h = float(new_h)
    
    new_exact_results = exact(x0, X)
    new_euler_results = euler_method(x0, X, y0, h)
    new_improved_euler_results = improved_euler_method(x0, X, y0, h)
    new_runge_kutta_results = runge_kutta(x0, X, y0, h)
    
    new_euler_error1 = []
    new_euler_error2 = []
    new_euler_error3 = []
    new_improved_euler_error1 = []
    new_improved_euler_error2 = []
    new_improved_euler_error3 = []
    new_runge_kutta_error1 = []
    new_runge_kutta_error2 = []
    new_runge_kutta_error3 = []
    for i in range(len(new_euler_results[0])):
        new_euler_error1.append(abs(count_exact(new_euler_results[0][i])-new_euler_results[1][i]))
        new_improved_euler_error1.append(abs(count_exact(new_euler_results[0][i])-new_improved_euler_results[1][i]))
        new_runge_kutta_error1.append(abs(count_exact(new_euler_results[0][i])-new_runge_kutta_results[1][i]))
    for i in range(len(new_euler_results[2])):
        new_euler_error2.append(abs(count_exact(new_euler_results[2][i])-new_euler_results[3][i]))
        new_improved_euler_error2.append(abs(count_exact(new_euler_results[2][i])-new_improved_euler_results[3][i]))
        new_runge_kutta_error2.append(abs(count_exact(new_euler_results[2][i])-new_runge_kutta_results[3][i]))
    for i in range(len(new_euler_results[4])):
        new_euler_error3.append(abs(count_exact(new_euler_results[4][i])-new_euler_results[5][i]))
        new_improved_euler_error3.append(abs(count_exact(new_euler_results[4][i])-new_improved_euler_results[5][i]))
        new_runge_kutta_error3.append(abs(count_exact(new_euler_results[4][i])-new_runge_kutta_results[5][i]))
    
    exact_dict1 = {'x': new_exact_results[0], 'y': new_exact_results[1]}
    exact_dict2 = {'x': new_exact_results[2], 'y': new_exact_results[3]}
    exact_dict3 = {'x': new_exact_results[4], 'y': new_exact_results[5]}
    
    euler_dict1 = {'x': new_euler_results[0], 'y': new_euler_results[1]}
    euler_dict2 = {'x': new_euler_results[2], 'y': new_euler_results[3]}
    euler_dict3 = {'x': new_euler_results[4], 'y': new_euler_results[5]}
   
    improved_dict1 = {'x': new_improved_euler_results[0], 'y': new_improved_euler_results[1]}
    improved_dict2 = {'x': new_improved_euler_results[2], 'y': new_improved_euler_results[3]}
    improved_dict3 = {'x': new_improved_euler_results[4], 'y': new_improved_euler_results[5]}
    
    runge_dict1 = {'x': new_runge_kutta_results[0], 'y': new_runge_kutta_results[1]}
    runge_dict2 = {'x': new_runge_kutta_results[2], 'y': new_runge_kutta_results[3]}
    runge_dict3 = {'x': new_runge_kutta_results[4], 'y': new_runge_kutta_results[5]}
     
    err_euler_dict1 = {'x': new_euler_results[0], 'y': new_euler_error1}
    err_euler_dict2 = {'x': new_euler_results[2], 'y': new_euler_error2}
    err_euler_dict3 = {'x': new_euler_results[4], 'y': new_euler_error3}
   
    err_improved_dict1 = {'x': new_improved_euler_results[0], 'y': new_improved_euler_error1}
    err_improved_dict2 = {'x': new_improved_euler_results[2], 'y': new_improved_euler_error2}
    err_improved_dict3 = {'x': new_improved_euler_results[4], 'y': new_improved_euler_error3}
    
    err_runge_dict1 = {'x': new_runge_kutta_results[0], 'y': new_runge_kutta_error1}
    err_runge_dict2 = {'x': new_runge_kutta_results[2], 'y': new_runge_kutta_error2}
    err_runge_dict3 = {'x': new_runge_kutta_results[4], 'y': new_runge_kutta_error3}
    
    exact1.data_source.data = exact_dict1
    exact2.data_source.data = exact_dict2
    exact3.data_source.data = exact_dict3
       
    euler1.data_source.data = euler_dict1
    euler2.data_source.data = euler_dict2
    euler3.data_source.data = euler_dict3
    
    improved_euler1.data_source.data = improved_dict1
    improved_euler2.data_source.data = improved_dict2
    improved_euler3.data_source.data = improved_dict3
    
    runge_kutta1.data_source.data = runge_dict1
    runge_kutta2.data_source.data = runge_dict2
    runge_kutta3.data_source.data = runge_dict3
    
    euler_err1.data_source.data = err_euler_dict1
    euler_err2.data_source.data = err_euler_dict2
    euler_err3.data_source.data = err_euler_dict3
    
    improved_euler_err1.data_source.data = err_improved_dict1
    improved_euler_err2.data_source.data = err_improved_dict2
    improved_euler_err3.data_source.data = err_improved_dict3
    
    runge_kutta_err1.data_source.data = err_runge_dict1
    runge_kutta_err2.data_source.data = err_runge_dict2
    runge_kutta_err3.data_source.data = err_runge_dict3
    
    push_notebook()

In [13]:
show(row([fig, err]),notebook_handle=True)


In [14]:
interact(update, x0='0', y0='3' , X='5.5', h='0.1')

interactive(children=(Text(value='0', description='new_x0'), Text(value='3', description='new_y0'), Text(value…

<function __main__.update(new_x0='0', new_y0='3', new_X='5.5', new_h='0.1')>